In [ ]:
import socket
import struct

# -----------------------------
# 1) UDP 监听配置
# -----------------------------
UDP_IP   = "0.0.0.0"   # 本机监听地址(所有网卡)
UDP_PORT = 10087

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

# -----------------------------
# 2) 数据格式
# -----------------------------
# 发送端约定：77 个 double（小端序），共 616 字节
EXPECTED_LEN   = 77
EXPECTED_BYTES = EXPECTED_LEN * 8
fmt = "<" + "d" * EXPECTED_LEN  # 小端序 + 77 个 double

# 字段名（列名）与发送端顺序对应
col_names = [
    "Time",            # values[0]  = sim_time
    "y_spcktime",      # values[1]
    "y_cb_vx",         # values[2]
    "y_cb_x",          # values[3]
    "y_cb_y",          # values[4]
    "y_cb_z",          # values[5]
    "y_cb_roll",       # values[6]
    "y_cb_yaw",        # values[7]
    "y_cb_pitch",      # values[8]
    "y_w01_rotw",      # values[9]
    "y_w02_rotw",      # values[10]
    "y_w03_rotw",      # values[11]
    "y_w04_rotw",      # values[12]
    "y_w05_rotw",      # values[13]
    "y_w06_rotw",      # values[14]
    "y_w07_rotw",      # values[15]
    "y_w08_rotw",      # values[16],
    "y_f01_x",         # values[17]
    "y_f01_y",         # values[18]
    "y_f01_z",         # values[19]
    "y_f01_roll",      # values[20]
    "y_f01_yaw",       # values[21]
    "y_f01_pitch",     # values[22]
    "y_f02_x",         # values[23]
    "y_f02_y",         # values[24]
    "y_f02_z",         # values[25]
    "y_f02_roll",      # values[26]
    "y_f02_yaw",       # values[27]
    "y_f02_pitch",     # values[28]
    "y_ws01_x",        # values[29]
    "y_ws01_y",        # values[30]
    "y_ws01_z",        # values[31]
    "y_ws01_roll",     # values[32]
    "y_ws01_yaw",      # values[33]
    "y_ws01_pitch",    # values[34]
    "y_ws02_x",        # values[35]
    "y_ws02_y",        # values[36]
    "y_ws02_z",        # values[37]
    "y_ws02_roll",     # values[38]
    "y_ws02_yaw",      # values[39]
    "y_ws02_pitch",    # values[40]
    "y_ws03_x",        # values[41]
    "y_ws03_y",        # values[42]
    "y_ws03_z",        # values[43]
    "y_ws03_roll",     # values[44]
    "y_ws03_yaw",      # values[45]
    "y_ws03_pitch",    # values[46]
    "y_ws04_x",        # values[47]
    "y_ws04_y",        # values[48]
    "y_ws04_z",        # values[49]
    "y_ws04_roll",     # values[50]
    "y_ws04_yaw",      # values[51]
    "y_ws04_pitch",    # values[52]
    "y_w01_rota",      # values[53]
    "y_w02_rota",      # values[54]
    "y_w03_rota",      # values[55]
    "y_w04_rota",      # values[56]
    "y_w05_rota",      # values[57]
    "y_w06_rota",      # values[58]
    "y_w07_rota",      # values[59]
    "y_w08_rota",      # values[60]
    "y_bar01_pitch",   # values[61]
    "y_bar02_pitch",   # values[62]
    "y_bar03_pitch",   # values[63]
    "y_bar04_pitch",   # values[64]
    "y_bar05_pitch",   # values[65]
    "y_bar06_pitch",   # values[66]
    "y_bar07_pitch",   # values[67]
    "y_bar08_pitch",   # values[68]
    "y_ws01_vy",       # values[69]
    "y_ws02_vy",       # values[70]
    "y_ws03_vy",       # values[71]
    "y_ws04_vy",       # values[72]
    "y_ws01_vyaw",     # values[73]
    "y_ws02_vyaw",     # values[74]
    "y_ws03_vyaw",     # values[75]
    "y_ws04_vyaw",     # values[76]
]

# -----------------------------
# 3) 缓存与文件输出逻辑
# -----------------------------
all_data = []        # 用于缓存全部行
threshold = 49.99    # 当 Time(=sim_time) >= 49.99 时，写出并退出
got_threshold = False

print(f"Listening on {UDP_IP}:{UDP_PORT} ...")

while True:
    data, addr = sock.recvfrom(65535)
    if len(data) != EXPECTED_BYTES:
        print(f"[WARN] Received {len(data)} bytes, expected {EXPECTED_BYTES} -> skipping")
        continue

    # 解析出 77 个 double
    values = struct.unpack(fmt, data)

    # 缓存该帧
    all_data.append(values)

    # 检查 Time(=values[0]) 是否超过 49.99
    if values[0] >= threshold and not got_threshold:
        got_threshold = True
        print(f"[INFO] sim_time >= {threshold}. Writing results to 'Result_UDPY_RosRt.log'...")

        # ---------------------------
        # 4) 写文件
        # ---------------------------
        with open("Result_UDPY_RosRt.log", "w") as f:
            # 写表头(带引号，用\t分隔)
            header_line = "\t".join(f'"{col}"' for col in col_names)
            f.write(header_line + "\n")

            # 写每行数据(按顺序输出，与col_names对应)
            for row in all_data:
                # row 是一个包含 77 个 float/double 的 tuple
                # 这里统一格式化为 6 位小数，可自行调整
                row_str = "\t".join(f"{v:.6f}" for v in row)
                f.write(row_str + "\n")

        print("[INFO] Done. Exiting script.")
        break  # 退出脚本（若需继续监听可改为 pass 或设别的逻辑）


# UDP数据采集程序增强版

增加了一个基于时间的监测机制（time-based monitoring mechanism）

现在该程序不仅会在仿真时间（simulation time）达到49.99秒时保存数据，还会在1分钟内没有接收到新数据时自动终止接收并保存数据。

- 引入time模块以实现精确的时间戳监控
- 设置socket超时参数，实现定期检查机制
- 添加数据接收超时逻辑，实现无数据自动退出功能
- 重构代码以提高可维护性，将文件写入逻辑封装为单独函数

## 超时检测机制（Timeout Detection Mechanism）：

- 使用time.time()记录最后一次数据接收时间
- 设置sock.settimeout(1.0)使socket每秒返回一次，以便检查整体超时


## 条件退出逻辑（Conditional Exit Logic）：

- 仿真时间阈值：sim_time >= 49.99
- 数据接收超时：current_time - last_receive_time > 60且已收集了数据

In [ ]:
import socket
import struct
import time  # 用于时间计算

# -----------------------------
# 1) UDP 监听配置
# -----------------------------
UDP_IP   = "0.0.0.0"   # 本机监听地址(所有网卡)
UDP_PORT = 10088

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

# 为了实现超时功能，设置socket超时
sock.settimeout(1.0)  # 1秒超时，允许定期检查时间

# -----------------------------
# 2) 数据格式
# -----------------------------
# 发送端约定：77 个 double（小端序），共 616 字节
EXPECTED_LEN   = 77
EXPECTED_BYTES = EXPECTED_LEN * 8
fmt = "<" + "d" * EXPECTED_LEN  # 小端序 + 77 个 double

# 字段名（列名）与发送端顺序对应
col_names = [
    "Time",            # values[0]  = sim_time
    "y_spcktime",      # values[1]
    "y_cb_vx",         # values[2]
    "y_cb_x",          # values[3]
    "y_cb_y",          # values[4]
    "y_cb_z",          # values[5]
    "y_cb_roll",       # values[6]
    "y_cb_yaw",        # values[7]
    "y_cb_pitch",      # values[8]
    "y_w01_rotw",      # values[9]
    "y_w02_rotw",      # values[10]
    "y_w03_rotw",      # values[11]
    "y_w04_rotw",      # values[12]
    "y_w05_rotw",      # values[13]
    "y_w06_rotw",      # values[14]
    "y_w07_rotw",      # values[15]
    "y_w08_rotw",      # values[16],
    "y_f01_x",         # values[17]
    "y_f01_y",         # values[18]
    "y_f01_z",         # values[19]
    "y_f01_roll",      # values[20]
    "y_f01_yaw",       # values[21]
    "y_f01_pitch",     # values[22]
    "y_f02_x",         # values[23]
    "y_f02_y",         # values[24]
    "y_f02_z",         # values[25]
    "y_f02_roll",      # values[26]
    "y_f02_yaw",       # values[27]
    "y_f02_pitch",     # values[28]
    "y_ws01_x",        # values[29]
    "y_ws01_y",        # values[30]
    "y_ws01_z",        # values[31]
    "y_ws01_roll",     # values[32]
    "y_ws01_yaw",      # values[33]
    "y_ws01_pitch",    # values[34]
    "y_ws02_x",        # values[35]
    "y_ws02_y",        # values[36]
    "y_ws02_z",        # values[37]
    "y_ws02_roll",     # values[38]
    "y_ws02_yaw",      # values[39]
    "y_ws02_pitch",    # values[40]
    "y_ws03_x",        # values[41]
    "y_ws03_y",        # values[42]
    "y_ws03_z",        # values[43]
    "y_ws03_roll",     # values[44]
    "y_ws03_yaw",      # values[45]
    "y_ws03_pitch",    # values[46]
    "y_ws04_x",        # values[47]
    "y_ws04_y",        # values[48]
    "y_ws04_z",        # values[49]
    "y_ws04_roll",     # values[50]
    "y_ws04_yaw",      # values[51]
    "y_ws04_pitch",    # values[52]
    "y_w01_rota",      # values[53]
    "y_w02_rota",      # values[54]
    "y_w03_rota",      # values[55]
    "y_w04_rota",      # values[56]
    "y_w05_rota",      # values[57]
    "y_w06_rota",      # values[58]
    "y_w07_rota",      # values[59]
    "y_w08_rota",      # values[60]
    "y_bar01_pitch",   # values[61]
    "y_bar02_pitch",   # values[62]
    "y_bar03_pitch",   # values[63]
    "y_bar04_pitch",   # values[64]
    "y_bar05_pitch",   # values[65]
    "y_bar06_pitch",   # values[66]
    "y_bar07_pitch",   # values[67]
    "y_bar08_pitch",   # values[68]
    "y_ws01_vy",       # values[69]
    "y_ws02_vy",       # values[70]
    "y_ws03_vy",       # values[71]
    "y_ws04_vy",       # values[72]
    "y_ws01_vyaw",     # values[73]
    "y_ws02_vyaw",     # values[74]
    "y_ws03_vyaw",     # values[75]
    "y_ws04_vyaw",     # values[76]
]

# -----------------------------
# 3) 缓存与文件输出逻辑
# -----------------------------
all_data = []        # 用于缓存全部行
threshold = 49.99    # 当 Time(=sim_time) >= 49.99 时，写出并退出
got_threshold = False

# 超时相关变量
timeout_seconds = 60  # 1分钟 = 60秒
last_receive_time = time.time()  # 初始化最后接收时间为当前时间

# 新增：数据接收计数器相关变量
data_count = 0
first_data_received = False
counter_interval = 2000  # 每1000次输出一次时间戳

print(f"Listening on {UDP_IP}:{UDP_PORT} ...")
print(f"Will exit after sim_time >= {threshold} or if no data received for {timeout_seconds} seconds")

# 写入日志文件
def write_log_file():
    print(f"[INFO] Writing results to 'Result_UDPY_RosRt.log'...")
    with open("Result_UDPY_RosRt.log", "w") as f:
        # 写表头(带引号，用\t分隔)
        header_line = "\t".join(f'"{col}"' for col in col_names)
        f.write(header_line + "\n")

        # 写每行数据(按顺序输出，与col_names对应)
        for row in all_data:
            # row 是一个包含 77 个 float/double 的 tuple
            # 这里统一格式化为 6 位小数，可自行调整
            row_str = "\t".join(f"{v:.6f}" for v in row)
            f.write(row_str + "\n")
    print("[INFO] File written successfully.")

while True:
    try:
        current_time = time.time()
        # 检查是否超时（距离上次接收数据超过1分钟）
        if current_time - last_receive_time > timeout_seconds and len(all_data) > 0:
            print(f"[INFO] No data received for {timeout_seconds} seconds.")
            write_log_file()
            break

        # 尝试接收数据
        data, addr = sock.recvfrom(65535)
        
        # 收到数据，更新最后接收时间
        last_receive_time = time.time()
        
        if len(data) != EXPECTED_BYTES:
            print(f"[WARN] Received {len(data)} bytes, expected {EXPECTED_BYTES} -> skipping")
            continue

        # 解析出 77 个 double
        values = struct.unpack(fmt, data)

        # 缓存该帧
        all_data.append(values)
        
        # 增加计数器
        data_count += 1
        
        # 第一次收到数据时提示
        if not first_data_received:
            print("[INFO] First data packet received.")
            first_data_received = True
        
        # 每1000次接收数据时输出时间戳
        if data_count % counter_interval == 0:
            print(f"[INFO] Received {data_count} packets. Current sim_time: {values[0]:.6f}")

        # 检查 Time(=values[0]) 是否超过 49.99
        if values[0] >= threshold and not got_threshold:
            got_threshold = True
            print(f"[INFO] sim_time >= {threshold}.")
            write_log_file()
            break  # 退出脚本

    except socket.timeout:
        # socket超时，不做特殊处理，循环将继续并检查是否达到1分钟无数据超时
        pass
    except Exception as e:
        print(f"[ERROR] {e}")
        if len(all_data) > 0:
            write_log_file()
        break

print("[INFO] Exiting script.")

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import font_manager
import os

def read_large_log_file(file_path):
    # 使用chunksize参数来分块读取大文件
    # 设置sep='\t'因为数据是用制表符分隔的
    # 使用float_precision='high'来保持高精度数值
    chunks = pd.read_csv(
        file_path,
        sep='\t',
        float_precision='high',
        chunksize=10000  # 每次读取10000行
    )
    
    # 初始化一个空的DataFrame来存储结果
    df = pd.DataFrame()
    
    # 分块处理数据
    for chunk in chunks:
        # 这里可以对每个chunk进行处理
        # 例如：只保留某些列，或者进行一些计算
        df = pd.concat([df, chunk], ignore_index=True)
        
    return df

# 读取自编 C++ 代码与 SIMPACK 实时联合仿真的结果文件
file_path = os.path.join(os.getcwd(), 'Result_UDPY_RosRt.log')

try:
    # 读取数据
    df = read_large_log_file(file_path)
    
    # 显示基本信息
    print("基于 UDP 跨机通信的 SIMPACK Realtime Log 文件基本信息：")
    print(f"行数: {len(df)}")
    print(f"列数: {len(df.columns)}")

    # # 显示数据基本统计信息
    # print("\n数据统计信息：")
    # print(df.describe())

except FileNotFoundError:
    print(f"错误：找不到文件 {file_path}")
except Exception as e:
    print(f"发生错误：{str(e)}")

# 创建图形
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df['Time'], 1000 * df['y_ws01_y'], 'b-', linewidth=1, label='Wheelset 01 LatDisp')
ax.plot(df['Time'], 1000 * df['y_ws02_y'], 'k-', linewidth=1, label='Wheelset 02 LatDisp')
ax.plot(df['Time'], 1000 * df['y_ws03_y'], 'y-', linewidth=1, label='Wheelset 03 LatDisp')
ax.plot(df['Time'], 1000 * df['y_ws04_y'], 'm-', linewidth=1, label='Wheelset 04 LatDisp')

ax.set_title('Wheelset Lateral Displacement', fontsize=14, pad=15)
ax.set_xlabel('Time [s]', fontsize=12)
ax.set_ylabel('Lateral Displacement [mm]', fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)
ax.legend(loc='upper right')
ax.tick_params(axis='both', which='major', labelsize=10)
plt.tight_layout()
plt.show()